In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/santander-customer-transaction-prediction/sample_submission.csv
/kaggle/input/santander-customer-transaction-prediction/train.csv
/kaggle/input/santander-customer-transaction-prediction/test.csv


In [2]:
train=pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/train.csv')
test=pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/test.csv')

In [3]:
train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [4]:
train.shape

(200000, 202)

In [5]:
train.columns

Index(['ID_code', 'target', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4',
       'var_5', 'var_6', 'var_7',
       ...
       'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195',
       'var_196', 'var_197', 'var_198', 'var_199'],
      dtype='object', length=202)

In [6]:
train['target'].value_counts()

target
0    179902
1     20098
Name: count, dtype: int64

In [7]:

train.isna().sum()*100/len(train)

ID_code    0.0
target     0.0
var_0      0.0
var_1      0.0
var_2      0.0
          ... 
var_195    0.0
var_196    0.0
var_197    0.0
var_198    0.0
var_199    0.0
Length: 202, dtype: float64

In [8]:
num_col=train.select_dtypes(exclude='object').columns.tolist()
len(num_col) # no object columns 

201

In [9]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split 
from sklearn import metrics 

In [10]:
X=train.drop(columns=['ID_code','target'])
y=train['target']


In [11]:
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=42)

In [12]:
X_train.shape, X_test.shape

((160000, 200), (40000, 200))

In [13]:
# Select the top 2 features using ANOVA F-test
selector = SelectKBest(score_func=f_classif, k=20)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# Get the selected feature names
selected_features = X.columns[selector.get_support()]
selected_features

Index(['var_2', 'var_6', 'var_12', 'var_13', 'var_21', 'var_22', 'var_26',
       'var_53', 'var_76', 'var_80', 'var_81', 'var_99', 'var_110', 'var_133',
       'var_139', 'var_146', 'var_165', 'var_166', 'var_174', 'var_190'],
      dtype='object')

In [14]:
X = X_train[selected_features]
#X.head(2)
X.shape

(160000, 20)

In [15]:
min_samples = min(len(X), len(y))
X = X[:min_samples]
y = y[:min_samples]


In [16]:
y.shape

(160000,)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.33,stratify=y, random_state=42)

In [18]:
!pip show h2o


Name: h2o
Version: 3.46.0.5
Summary: H2O, Fast Scalable Machine Learning, for python 
Home-page: https://github.com/h2oai/h2o-3.git
Author: H2O.ai
Author-email: support@h2o.ai
License: Apache v2
Location: /opt/conda/lib/python3.10/site-packages
Requires: requests, tabulate
Required-by: 


In [19]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.24" 2024-07-16; OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu322.04); OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu322.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp311wkllh
  JVM stdout: /tmp/tmp311wkllh/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp311wkllh/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,1 month and 28 days
H2O_cluster_name:,H2O_from_python_unknownUser_yknmed
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.250 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [20]:
train_h2o = h2o.H2OFrame(pd.concat([X_train, y_train], axis=1))
X_val = h2o.H2OFrame(X_val)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [27]:
train_h2o = train_h2o.na_omit()


In [28]:
print(train_h2o['target'].isna().sum())


0.0


In [29]:
target = 'target'
features = X.columns.tolist()

In [30]:
train_h2o[target] = train_h2o[target].asfactor()

In [31]:
automl = H2OAutoML(
        max_models = 20,
        seed = 42,
        sort_metric = 'AUC',
        include_algos = ["GBM", "XGBoost", "GLM", "StackedEnsemble", "DRF"],
        balance_classes = True,
        nfolds = 5)

In [32]:
automl.train(x=features, y=target, training_frame=train_h2o)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),9/20
# GBM base models (used / total),1/8
# XGBoost base models (used / total),7/9
# GLM base models (used / total),1/1
# DRF base models (used / total),0/2
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [37]:
print(test.columns)



Index(['ID_code', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5',
       'var_6', 'var_7', 'var_8',
       ...
       'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195',
       'var_196', 'var_197', 'var_198', 'var_199'],
      dtype='object', length=201)


In [38]:
X_test = test.drop(columns=['ID_code'])

In [39]:
X_test= X_test[selected_features]

In [40]:
X_test

,var_2,var_6,var_12,var_13,var_21,var_22,var_26,var_53,var_76,var_80,var_81,var_99,var_110,var_133,var_139,var_146,var_165,var_166,var_174,var_190
0,12.9536,5.8493,13.9696,0.3458,18.8064,1.5899,-9.4902,7.3213,7.3075,9.4883,16.9060,1.8585,3.7662,7.5147,-2.5502,11.6793,22.9530,2.5531,18.1317,-2.1556
1,11.3047,6.0196,14.1129,2.5667,3.1389,5.2578,-5.2341,6.8481,6.9388,1.9772,14.0406,-1.4166,1.7375,7.0891,6.7530,8.7644,15.5134,3.2185,10.5018,10.6165
2,10.1407,4.8950,13.6751,3.8183,12.3696,4.7473,-15.4246,5.6510,-9.7077,16.1691,14.3299,1.7818,8.6896,6.5950,-6.0452,11.6749,13.3009,3.1243,27.4035,-0.7484
3,12.0220,4.9397,14.0526,13.5010,12.5527,2.9707,1.0061,6.2536,12.5465,17.5941,15.4375,2.6165,10.3445,6.7888,12.7898,8.6963,24.8712,2.6415,14.5499,9.5702
4,14.1295,6.8595,14.1013,8.9672,16.3059,6.7954,8.6228,5.6794,8.1522,11.8133,16.7661,0.1148,6.3933,7.1434,11.7989,12.9327,26.2040,3.2348,14.0037,4.2259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,10.4333,4.8579,14.3201,17.4594,9.8526,4.0207,-12.4547,6.4420,7.6126,7.7879,13.9172,-2.3265,1.9588,6.9553,14.0376,8.0605,21.8848,3.5995,26.2206,2.0544
199996,7.3443,5.6888,13.8246,6.6547,16.6921,5.5064,0.0592,6.2642,7.8092,12.1609,15.5275,-2.0057,6.3677,6.6422,16.9374,9.9044,18.2834,3.0946,20.1379,5.0071
199997,11.2074,5.3883,14.0675,13.9975,16.0286,0.6066,-9.1787,6.2427,-4.1993,12.0411,15.2880,-0.3975,0.2553,6.7956,16.5629,12.0198,11.9434,3.2239,18.9432,5.1536
199998,13.6584,4.1959,14.3051,4.2644,20.9599,1.1101,3.6157,4.9431,15.5314,2.3608,9.1805,0.3056,10.8259,7.0122,7.2125,10.3614,20.1410,3.2078,13.2572,3.4259


In [41]:
test_h2o = h2o.H2OFrame(X_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [42]:
best_model = automl.leader
predictions = best_model.predict(test_h2o)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [43]:
output = predictions.as_data_frame()
output

/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


,predict,p0,p1
0,0,0.837023,0.162977
1,0,0.867935,0.132065
2,0,0.859046,0.140954
3,0,0.927896,0.072104
4,0,0.890644,0.109356
...,...,...,...
199995,0,0.982940,0.017060
199996,0,0.967539,0.032461
199997,0,0.981287,0.018713
199998,0,0.884084,0.115916


In [49]:
print(len(submission), len(predictions))


200000 200000


In [51]:
submission = pd.read_csv("/kaggle/input/santander-customer-transaction-prediction/sample_submission.csv")

# Assuming 'predictions' is already a pandas DataFrame at this stage
# Apply threshold and assign to submission file
submission['target'] = (predictions['p1'] > 0.5).astype(int)

# Display the head of the submission DataFrame
submission.head()


,ID_code,target
0,test_0,0
1,test_1,0
2,test_2,0
3,test_3,0
4,test_4,0


In [52]:
submission.to_csv('submission.csv', index=False)